# <span style="color: blue;">Функции</span>

Имя функции -- типично (буквы, цифры, подчёркивания, не может начинаться с цифры)

В Python 3.x можно использовать любые языки (но не стоит)

In [ ]:
def foo():
    return 42
foo()

`return` можно не использовать, по умолчанию возвращается `None`

`return` без аргумента тоже возвращает `None`

In [ ]:
def empty():
    pass

print(empty())

Любая функция в Python -- это именно функция, а не процедура

А также любая функция в Python -- это объект. И поэтому их можно присваивать переменным.

In [ ]:
bar = foo
bar()

#### Документирование

Указывается первой строкой, а не комментариями (как в других языках)

Благодаря этому к ним можно получить доступ во время выполнения программы

In [ ]:
def foo():
    """ Первой строкой """
    return 42

foo.__doc__

Можно также получить:
* в интерпретаторе через: `help(foo)`
* в ipython через: `? foo`

Есть специальные утилиты для генерации документации

#### Простой пример

In [ ]:
def min(x, y):
    return x if x < y else y

In [ ]:
min(-5, 12)

In [ ]:
min(x=-5, y=12)

In [ ]:
min(y=12, x=-5)

In [ ]:
min(x=-5, z=12)

#### План
* произвольное число аргументов
* min для кортежей, списков, множеств и других последовательностей
* ограничить минимум произвольным отрезком [lo, hi]
* по lo и hi строить bounded_min (фабрика функций)

### Упаковка аргументов

In [ ]:
def min(*args):  # type(args) == tuple
    res = float("inf")  # бесконечность, целые числа с ней сравниваются корректно
    for arg in args:
        if arg < res:
            res = arg
    return res
min(-5, 12, 13)

In [ ]:
min()

Как потребовать, чтобы в args был хотя бы один элемент?

In [ ]:
def min(first, *args):
    res = first
    for arg in args:
        if arg < res:
            res = arg
    return res

min()

#### Немного о бесконечности

In [ ]:
10 * 1000 < float('inf')

In [ ]:
import math
float('inf') == math.inf

In [ ]:
type(math.inf)

In [ ]:
math.inf + 2 == math.inf

In [ ]:
math.inf - math.inf

Как применить функцию min к коллекции?

In [ ]:
xs = {-5, 12, 13}
# min(??)

### Распаковка аргументов
* будет работать с любым объектом, поддерживающим протокол итератора

In [ ]:
xs = {-5, 12, 13}
min(*xs)  # но нужно иметь в виду, что для set нет определённого порядка

In [ ]:
min(*[-5, 12, 13])

In [ ]:
min(*(-5, 12, 13))

Об итераторах потом, а пока вспомним про `bounded_min`

In [ ]:
bounded_min(-5, 12, 13, lo=0, hi=255)
# как?

### Ключевые аргументы: аргументы по умолчанию

Чтобы не делать несколько перегруженных функций

In [ ]:
def bounded_min(first, *args, lo=float("-inf"), hi=float("inf")):
    res = hi
    for arg in (first, ) + args:
        if arg < res and lo <= arg < hi:
            res = arg
    return max(res, lo)

Можно писать и так: 10 > arg > 4, но это некрасиво, "йода-условие" :)

Когда происходит инициализация ключевых аргументов со значениями по умолчанию?

Ответ: в момент определения функции, и вычисляется лишь единожды

In [ ]:
i = 5
def f(a1, a2=i):
    print(a1, a2)
i = 6
f(1)

In [ ]:
f(1, 2)

In [ ]:
f(a2=2, 1)  # Ошибка: именованные должны быть после позиционных

#### Подводные камни

In [ ]:
def unique(iterable, seen=set()):
    acc = []
    for item in iterable:
        if item not in seen:
            seen.add(item)
            acc.append(item)
    return acc

xs = [1, 1, 2, 3]
unique(xs)

In [ ]:
# хороший пример
seen = {2}
print(unique(xs, seen))
print(seen)

In [ ]:
# хороший пример
s = set()
unique(xs, s)
s

In [ ]:
# плохой пример:
unique(xs)
unique(xs)

In [ ]:
unique.__defaults__

Все вызовы `unique` шарят параметры по умолчанию между собой

In [ ]:
# Ещё простой пример:
def f(value, lst=[]):
    lst.append(value)
    return lst

print(f(1))
print(f(2))

Изменяемые значения лучше не использовать в качестве значений по умолчанию

### Как правильно инициализировать?

In [ ]:
def unique(iterable, seen=None):
    seen = set(seen or [])  # None -- falsy.
    # копируем, потому что вызывающая сторона может не ожидать, что set может быть изменён внутри
    acc = []
    for item in iterable:
        if item not in seen:
            seen.add(item)
            acc.append(item)
    return acc
xs = [1, 1, 2, 3]
unique(xs)

In [ ]:
unique(xs)

In [ ]:
# Простой пример:
def f(value, lst=None):
    if lst is None:
        lst = []
    lst.append(value)
    return lst

print(f(1))
print(f(2))

In [ ]:
# falsy-значения (там где ожидается bool, они будут False):
0, 0.0, 0j
""
[], (), {}, set()
None
# Всё остальное True

### Только ключевые (в v3)

In [ ]:
def flatten(xs, depth=None):
    pass
# расплющивает списки с произвольной вложенностью не до самого конца, а до определённой глубины depth
flatten([1, [2], 3], depth=1)
flatten([1, [2], 3], 1)

def flatten(xs, *, depth=None):
    pass # обязываем использовать параметр depth только по имени
# * -- передать позиционные нельзя, просто показывается, что они как бы все скушаны, но принять никто не может :)

### Упаковка и распаковка ключевых аргументов

In [ ]:
def runner(cmd, **kwargs):
    if kwargs.get("verbose", True):
        print("Logging enabled")
runner("mysqld", limit=42)

In [ ]:
runner("mysqld", **{"verbose": False})

In [ ]:
#... func(*args, **kwargs)

In [ ]:
options = {"verbose": False}
runner("mysqld", **options)

Лучше называйте стандартно:
`*args`,
`**kwargs` или `**kw`

### Распаковка и присваивание

In [ ]:
acc = []
seen = set()
(acc, seen) = ([], set())
x = 1
y = 2
x, y = y, x
x, y = y, x + y

In [ ]:
tp = (1, 2, 3)
a, b, c = tp
a

In [ ]:
# справа может быть любой объект, поддерживающий протокол итератора
x, y, z = [1, 2, 3]
x, y, z = {1, 2, 3}  # unordered!
x, y, z = "xyz"
x, y, z

In [ ]:
# Скобки обычно опускают, но иногда они бывают полезны
rectangle = (0, 0), (4, 4)
(x1, y1), (x2, y2) = rectangle
# распаковка слева работает рекурсивно :)

In [ ]:
# можно использовать подчёркивание для ненужных переменных
# но лучше после подчёркивания давать имя
lst = [1, 2, 3, 4]
_, _, z, _ = lst

tpl = (255, 255, 0, 100)
_r, g, b, _a = lst

### Расширенный синтаксис распаковки (v3.0)

In [ ]:
# расширенный синтаксис
first, *rest = range(1, 5)
first  # 1

In [ ]:
rest  # [2, 3, 4]

In [ ]:
# * может быть только одна
first, *rest, last = range(1, 5)
last  # 4
# может быть полезно, если это не список, а нечто по чему можно итерироваться, например, файл

In [ ]:
a, *b, c = [1, 2]
b

In [ ]:
first, *rest, last = [42]  # если не хватает

In [ ]:
# Выражения с * могут быть рекурсивными:
*_, (first, *rest) = [range(1, 5)] * 5
first  # 1

In [ ]:
*_, (first, *_) = [range(1, 5), range(2, 5), range(3, 5)]
first  # 3

In [ ]:
def f():
    return 10, 'file.txt'

length, path = f()

### Распаковка и цикл for

In [ ]:
for a, *b in [range(4), range(2)]:
    print(b)

In [ ]:
for a, b in [(1, 2), (3, 4)]:
    pass

In [ ]:
lst = [1, 2, 3, 4, 5, 'test']
for i, b in enumerate(lst):
    pass

In [ ]:
dict2 = {'key1': 1, 'key2': 2}
for key, value in dict2.items():
    print(key, value)

In [ ]:
list(dict2.values())

In [ ]:
for i, value in enumerate(dict2.values()):
    print(i, value)

In [ ]:
lst1 = range(5)
lst2 = ['a', 'b', 'c']

for a, b in zip(lst1, lst2):  # как молнией (на одежде) соединем два списка
    pass

### Распаковка и байткод

In [ ]:
import dis
dis.dis("first, *rest, last = ('a', 'b', 'c')")

In [ ]:
# Мораль: Присваивание в Python работает слева направо
x, (x, y) = 1, (2, 3)
x

In [ ]:
dis.dis("first, *rest, last = ['a', 'b', 'c']")

Почти похоже, но не совсем

**Мораль:** синтаксически схожие конструкции могут иметь различную семантику времени исполнения

### Дополнительные расширения синтаксиса распаковки

In [ ]:
# в 3.5 расширили

def f(*args, **kwargs):
    print(args, kwargs)
    
# При вызове функции можно распаковать несколько объектов:
f(1, 2, *[3, 4], *[5], foo="bar", **{"baz": 42}, boo=24)

In [ ]:
# и при инициализации контейнеров:
defaults = {"host": "0.0.0.0", "port": 8080}
{**defaults, "port": 80}

In [ ]:
[*range(5), 6]  # аналогично для множества и кортежа. только ничего не замещает

## Типы параметров

_TODO_